In [4]:
from requests.exceptions import RequestException
from contextlib import closing
import re
from lxml import html
import requests
from datetime import datetime
import Algorithmia
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"./climate_sentiment_key.json"

In [31]:
def get_headline(url, verbose=False):
    # download HTML from simplified page
    base_url = "https://www.textise.net/showText.aspx?strURL="
    if verbose: print("Downloading HTML page...")
    page = requests.get(base_url + url.replace(":", "%253A"))
    tree = html.fromstring(page.content)
    if verbose: print("Finished downloading. Parsing...")
    
    divs = []
    for div in tree.xpath('//div/text()'):
        divs.append(div.rstrip())
    # delete filler text
    delete = ["Español", 'Set edition preference:', u'\xa0 -', "International  Edition", "International Edition", "Find us on", "Read More", 'U.S.', "Arabic", 'International', 'Here are some options:', "Switzerland", " FOLLOW CNN BUSINESS "]
    divs = filter(lambda x: re.sub(' \d+ of \d+', '', x), divs)
    divs = filter(lambda x: x != [], divs)
    divs = filter(lambda x: x != "<div>", divs)
    divs = filter(lambda x: x != "</div>", divs)
    divs = filter(lambda x: x != "\xa0", divs)
    divs = filter(lambda x: x != ".", divs)
    divs = filter(lambda x: x != " |", divs)
    divs = filter(lambda x: x != "", divs)
    divs = filter(lambda x: "/>" not in x, divs)
    divs = filter(lambda x: "</" not in x, divs)
    divs = filter(lambda x: "http" not in x, divs)
    divs = filter(lambda x: "Image:" not in x, divs)
    divs = filter(lambda x: u"\xa0" not in x, divs)
    divs = filter(lambda x: "Facebook Messenger" not in x, divs)
    divs = filter(lambda x: "Hide Caption" not in x, divs)
    divs = filter(lambda x: "MUST WATCH" not in x, divs)
    divs = filter(lambda x: x not in delete, divs)
    divs = "".join(list(divs)[1:-4])
    sentences = [x.strip() for x in divs.split(".")][0].split('\n')[0]
    return sentences

In [32]:
repr(get_headline("http://money.cnn.com/2012/11/07/news/economy/climate-change-obama/index.html"))

"'Climate change is back on the table'"